In [1]:
import tensorflow as tf
import os
from tqdm import tqdm
import urllib.request as req

In [3]:
# ! pip install tqdm

In [5]:
SOURCE_URL = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip"

In [6]:
DESTINATION = "data"
os.makedirs(DESTINATION, exist_ok=True)
data_file = "data.zip"

DESTINATION_ZIP_PATH = os.path.join(DESTINATION, data_file)

In [7]:
# filename, headers = req.urlretrieve(SOURCE_URL, DESTINATION_ZIP_PATH)

In [8]:
filename

'data\\data.zip'

In [9]:
print(headers)

Content-Type: application/octet-stream
Accept-Ranges: bytes
Server: Windows-Azure-Blob/1.0 Microsoft-HTTPAPI/2.0
x-ms-blob-content-md5: l2WJyK3tt5/f2azyQ9eE8w==
Last-Modified: Sat, 13 Oct 2018 12:14:22 GMT
ETag: "0x8D6310568FD1DF6"
Content-Length: 824894548
Date: Fri, 08 Apr 2022 05:25:09 GMT
Connection: close




In [2]:
from zipfile import ZipFile

with ZipFile(filename, "r") as zip_f:
    zip_f.extractall("data/")

NameError: name 'filename' is not defined

In [2]:
TARGET_DATA_DIRS = ["Cat", "Dog"]
PARENT_DIR = os.path.join("data","PetImages")

In [3]:
# !pip install Pillow

In [4]:
from PIL import Image

In [5]:
BAD_DATA_DIR = "bad_data"
os.makedirs(BAD_DATA_DIR, exist_ok=True)

In [6]:
img = Image.open(r"C:\jay\Deep Learning classes\CNN_Classifier\notebook\data\PetImages\Dog\4.jpg")
img.format

'JPEG'

In [7]:
if img.format in ['JPEG', 'PNG', 'GIF', 'BMP']:
    print('hello')

hello


In [8]:
import shutil
import imghdr

In [9]:
for dirs in os.listdir(PARENT_DIR):
    full_path_data_dir = os.path.join(PARENT_DIR, dirs)
    for imgs in os.listdir(full_path_data_dir):
        path_to_img = os.path.join(full_path_data_dir, imgs)
        try:
            img = Image.open(path_to_img)
            img.verify()
                
#             if len(img.getbands()) !=3 or imghdr.what(path_to_img) not in ['jpeg','png']:
#                 bad_data_path = os.path.join(BAD_DATA_DIR, imgs)
#                 shutil.move(path_to_img, bad_data_path)
#                 continue
                
#             print(f"{path_to_img} is verified with format {imghdr.what(path_to_img)}")
        except Exception as e:
            print(f"{path_to_img} is bad !")
            bad_data_path = os.path.join(BAD_DATA_DIR, imgs)
            shutil.move(path_to_img, bad_data_path)

data\PetImages\Cat\666.jpg is bad !
data\PetImages\Cat\Thumbs.db is bad !
data\PetImages\Dog\11702.jpg is bad !


C:\jay\Deep Learning classes\CNN_Classifier\env\lib\site-packages\PIL\TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


data\PetImages\Dog\Thumbs.db is bad !


In [10]:
IMG_SIZE = (180, 180)
BATCH_SIZE = 32

In [11]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    PARENT_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 24998 files belonging to 2 classes.
Using 19999 files for training.
Found 24998 files belonging to 2 classes.
Using 4999 files for validation.


In [12]:
### Create a log dir for tensorboard logs

import time

def get_log_path(base_log_dir=os.path.join("logs","fit")):
    uniqueName = time.asctime().replace(" ", "_").replace(":", "")
    log_path = os.path.join(base_log_dir, uniqueName)
    print(f"saving logs at: {log_path}")
    return log_path

log_dir = get_log_path()

saving logs at: logs\fit\Fri_Apr__8_125333_2022


In [13]:
file_writer = tf.summary.create_file_writer(logdir=log_dir)

In [14]:
for images, labels in train_ds.take(1):
    print(images.shape, labels)

(32, 180, 180, 3) tf.Tensor([0 0 1 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 0 1 1 0 0 0 1 0 1 1 1], shape=(32,), dtype=int32)


In [15]:
import numpy as np

In [16]:
with file_writer.as_default():
    images = np.array(images) ### (20, 28, 28, 1)

    tf.summary.image("samples", images.astype("uint8"), max_outputs=25, step=0)